## Problema de Asignación con Datos Externos

Este ejemplo automatiza la creación de variables y restricciones utilizando listas/diccionarios de Python, lo que es esencial para modelos de la vida real.

Imagina que tienes 3 **Trabajadores** y 3 **Tareas**, y quieres minimizar el costo total.


| Costo   | Tarea 1 | Tarea 2 | Tarea 3 |
| ------- | ------- | ------- | ------- |
| Trab. 1 | 9       | 4       | 5       |
| Trab. 2 | 6       | 8       | 7       |
| Trab. 3 | 3       | 7       | 6       |


In [2]:
# Importamos la libreria gurobi con alias grb
import gurobipy as grb

In [ ]:
# Datos del problema
Trabajadores = ["T1", "T2", "T3"]
Tareas = ["Tarea1", "Tarea2", "Tarea3"]
Costos = {
    ("T1", "Tarea1"): 9,
    ("T1", "Tarea2"): 4,
    ("T1", "Tarea3"): 2,
    ("T2", "Tarea1"): 6,
    ("T2", "Tarea2"): 1,
    ("T2", "Tarea3"): 3,
    ("T3", "Tarea1"): 3,
    ("T3", "Tarea2"): 2,
    ("T3", "Tarea3"): 1,
}

In [ ]:
# Creamos el modelo
m = grb.Model("Asignación de datos Externos")

Set parameter Username
Set parameter LicenseID to value 2718415
Academic license - for non-commercial use only - expires 2026-10-06


In [5]:
# 1. Variables: x[i,j] es BINARY (1 si el trabajador i hace la tarea j, 0 sino)
# Usamos diccionario de tuplas para automatizar la creación de la matriz X
X = m.addVars(Trabajadores, Tareas, vtype=grb.GRB.BINARY, name="x")

In [ ]:
# Función Objetivo: Minimizar Costo Total (suma del costo * variable)
m.setObjective(
    grb.quicksum(X[i, j] for i in Trabajadores for j in Tareas), grb.GRB.MINIMIZE
)

# Restricciones
# Cada Tarea debe ser asignada a EXACTAMENTE un Trabajador
m.addConstrs(
    (grb.quicksum(X[i, j] for i in Trabajadores) == 1 for j in Tareas),
    name="UnTareaPorTrab",
)

# Cada Trabajador debe hacer a lo sumo UNA Tarea
m.addConstrs(
    (grb.quicksum(X[i, j] for j in Tareas) <= 1 for i in Trabajadores),
    name="UnTrabPorTarea",
)

# Optimizar
m.optimize()

# Mostrar Resultados
if m.status == grb.GRB.Status.OPTIMAL:
    print("\n--- Resultados (Intermedio) ---")
    print(f"Costo Mínimo Total: {m.ObjVal:.2f}\n")
    print("Asignaciones")
    # Filtrar solo las variables que son igual a 1
    asignaciones = X.keys()
    for i, j in asignaciones:
        if X[i, j].X > 0.5:  # Si el valor es 1 (asignado)
            print(f" -{i} asginado a {j} (Costo: {Costos[(i,j)]})")
        else:
            print("No se encontró solución óptima.")

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 12 rows, 9 columns and 36 nonzeros
Model fingerprint: 0x323d22e8
Variable types: 0 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded MIP start from previous solve with objective 3

Presolve removed 6 rows and 0 columns
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 18 nonzeros
Variable types: 0 continuous, 9 integer (9 binary)

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 12 (of 12 available processors)

Solution count 1: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000